In [73]:
import openpyxl
import pprint
import re
import pandas as pd

#Excel読み込み
wb1 = openpyxl.load_workbook("sample.xlsx")
wb2 = openpyxl.load_workbook("seikyu.xlsx")

#シートネーム全部リスト化
wb1sheet = [sheetname for sheetname in wb1.sheetnames]#請求元サンプル入ってる。
wb2sheet = [sheetname for sheetname in wb2.sheetnames]#請求書一覧データ


#3行目をカラムとしてリスト化
col = []
for row in sheet1.iter_cols():
    for cell in row:
        if cell.row == 3:
            col.append(cell.value)
print(col)

#各シートデータ、行最大値リスト化
sheet_list = []
max_row_list = []
for i,sheet in enumerate(wb2sheet):
    if re.search('請求一覧', sheet):
        sheet_row = wb2[wb2sheet[i]]
        sheet_list.append(sheet_row)
        max_row_list.append(sheet_row.max_row)

print(sheet_list)
print(max_row_list)


['受注No.', '相手先名', '相手先コード', '商品名１', '商品名２', '検収日', '金額（税抜き）', '備考']
[<Worksheet "請求一覧2101">, <Worksheet "請求一覧2102">, <Worksheet "請求一覧2103">, <Worksheet "請求一覧2105">, <Worksheet "請求一覧2106">, <Worksheet "請求一覧2107">, <Worksheet "請求一覧2108">, <Worksheet "請求一覧2109">, <Worksheet "請求一覧2110">]
[13, 13, 13, 13, 13, 13, 13, 13, 13]


In [58]:
#行情報取得関数（取得した行をリストに入れる）
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])


l_2d = get_value_list(sheet1)

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                          max_col=end_col))

l_2d = get_list_2d(sheet1, 4, max1, 0, 8)
#pprint.pprint(l_2d, width=40)

In [59]:
#取得したリストからデータフレーム作成
df = pd.DataFrame(l_2d)

dic_col = {}

#カラム辞書化
for i,col in enumerate(col):
    dic_col[i] = col

print(dic_col)
#列名置き換え
seikyu_df = df.rename(columns=dic_col)
seikyu_df = seikyu_df.dropna(how='all')

display(seikyu_df)

{0: '受注No.', 1: '相手先名', 2: '相手先コード', 3: '商品名１', 4: '商品名２', 5: '検収日', 6: '金額（税抜き）', 7: '備考'}


,受注No.,相手先名,相手先コード,商品名１,商品名２,検収日,金額（税抜き）,備考
0,201201.0,None,1005.0,顧問料,None,28.0,80000.0,2020.12月分
